<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

Has construido un modelo. ¿Pero qué tan bueno es?

En esta lección, aprenderás a utilizar la **validación de modelos** para medir la calidad de tu modelo. Medir la calidad del modelo es clave para mejorarlo de manera iterativa.

# ¿Qué es la Validación de Modelos?

Querrás evaluar prácticamente todos los modelos que construyas. En la mayoría de las aplicaciones (aunque no en todas), la métrica relevante de calidad del modelo es la **precisión predictiva**. En otras palabras, ¿las predicciones del modelo estarán cerca de lo que realmente ocurre?

Muchas personas cometen un error grave al medir la precisión predictiva: realizan predicciones sobre los *datos de entrenamiento* y las comparan con los valores objetivo en esos mismos datos de entrenamiento. Verás el problema de este enfoque y cómo resolverlo en un momento, pero primero pensemos cómo se haría esto inicialmente.

Primero necesitarías resumir la calidad del modelo de una manera comprensible. Si comparas los valores reales y predichos de 10,000 viviendas, probablemente encuentres una mezcla de buenas y malas predicciones. Revisar una lista de 10,000 predicciones y valores reales no tiene sentido. Necesitamos resumir eso en una sola métrica.

Existen muchas métricas para resumir la calidad de un modelo, pero comenzaremos con una llamada **Error Absoluto Medio** (también conocida como **MAE**, por sus siglas en inglés). Desglosaremos esta métrica comenzando por la última palabra: error.

El error de predicción para cada casa es:<br>

```
error=actual−predicted
```

Entonces, si una casa cuesta \$150,000 y se predijo que costaría \$100,000, el error es de \$50,000.

Con la métrica MAE, se toma el valor absoluto de cada error. Esto convierte cada error en un número positivo. Luego, se calcula el promedio de esos errores absolutos. Esta será nuestra medida de calidad del modelo. En términos sencillos, se puede decir como:

> En promedio, nuestras predicciones se desvían aproximadamente X.

Para calcular el MAE, primero necesitamos un modelo. Ese modelo se construye en una celda oculta que puedes revisar haciendo clic en el botón `code`.


In [2]:
import pandas as pd

# Cargar los datos
melbourne_file_path = '/content/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Filtrar las filas con valores faltantes en el precio
filtered_melbourne_data = melbourne_data.dropna(axis=0)

# Seleccionar la variable objetivo (target) y las características (features)
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea',
                      'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor

# Definir el modelo
melbourne_model = DecisionTreeRegressor()

# Ajustar (entrenar) el modelo
melbourne_model.fit(X, y)

DecisionTreeRegressor()

Una vez que tenemos un modelo, así es como calculamos el **error absoluto medio** (MAE, por sus siglas en inglés):

In [3]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

# El Problema de las Métricas "Dentro de la Muestra"

La medida que acabamos de calcular puede llamarse una métrica **dentro de la muestra** (*in-sample*). Utilizamos una única "muestra" de viviendas tanto para construir el modelo como para evaluarlo. Veamos por qué esto es problemático.

Imagina que, en el gran mercado inmobiliario, el color de la puerta no tiene ninguna relación con el precio de la vivienda.

Sin embargo, en la muestra de datos que utilizaste para construir el modelo, todas las casas con puertas verdes eran muy costosas. La tarea del modelo es encontrar patrones que predigan el precio de las viviendas, por lo tanto detectará este patrón y siempre predecirá precios altos para casas con puertas verdes.

Dado que este patrón fue derivado de los datos de entrenamiento, el modelo parecerá preciso en esos datos.

Pero si dicho patrón no se mantiene cuando el modelo ve nuevos datos, entonces será muy inexacto en la práctica.

Como el valor práctico de los modelos radica en hacer predicciones sobre datos nuevos, debemos medir el rendimiento en datos que **no** se usaron para construir el modelo. La forma más directa de hacer esto es excluir una parte de los datos durante el proceso de construcción del modelo, y luego utilizar esa parte para probar la precisión del modelo sobre datos que no ha visto antes. Estos datos se denominan **datos de validación**.


# Implementación en Código

La librería `scikit-learn` proporciona una función llamada `train_test_split` para dividir los datos en dos partes. Usaremos una parte de los datos como **datos de entrenamiento** para ajustar el modelo, y la otra parte como **datos de validación** para calcular el `mean_absolute_error`.

Aquí está el código:



In [4]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y conjunto de validación, tanto para características como para la variable objetivo
# La división se basa en un generador de números aleatorios. Proporcionar un valor numérico al argumento
# random_state garantiza que obtendremos la misma división cada vez que ejecutemos este script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# Definir el modelo
melbourne_model = DecisionTreeRegressor()

# Ajustar (entrenar) el modelo con los datos de entrenamiento
melbourne_model.fit(train_X, train_y)

# Obtener las predicciones de precios sobre los datos de validación
val_predictions = melbourne_model.predict(val_X)

# Imprimir el error absoluto medio de las predicciones sobre el conjunto de validación
print(mean_absolute_error(val_y, val_predictions))

262973.28147191735


# ¡Wow!

Tu error absoluto medio en los datos **dentro de la muestra** fue de aproximadamente 500 dólares. En los datos **fuera de la muestra** supera los 250,000 dólares.

Esta es la diferencia entre un modelo que es casi exactamente correcto y uno que resulta inutilizable para la mayoría de los propósitos prácticos. Como punto de referencia, el valor promedio de las viviendas en los datos de validación es de 1.1 millones de dólares. Por lo tanto, el error en los datos nuevos representa aproximadamente una cuarta parte del valor promedio de una vivienda.

Existen muchas formas de mejorar este modelo, como experimentar para encontrar mejores características (features) o probar con diferentes tipos de modelos.
